In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
train_features = pd.read_csv("train_features.csv")
test_features = pd.read_csv("test_features.csv")
train_labels = pd.read_csv("dataset/train_similar.csv")
test_labels = pd.read_csv("dataset/test_similar.csv")

In [3]:
train_features

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.275829,0.257495,0.067623,0.000000,0.118199,0.003383,0.051705,0.000000,0.134602,0.008163,...,0.000000,0.184349,0.116857,0.114778,0.023528,0.657772,0.291918,0.000615,0.000000,0.963517
1,0.948423,1.021655,0.450011,0.023558,0.651246,0.219234,0.355012,0.155480,0.796764,0.039686,...,0.019940,0.096511,0.000000,0.095244,0.141445,0.000000,0.154032,1.792427,0.340598,0.836082
2,0.952304,0.843574,0.366320,0.802187,0.095241,0.626198,0.592671,0.084656,0.259383,1.553256,...,0.903022,0.042275,0.572444,0.064828,1.332589,0.420124,1.281798,0.298741,0.181010,0.058992
3,0.066787,0.565224,0.422215,0.096929,0.052580,0.214827,0.620173,0.223013,0.390939,0.609273,...,1.710189,0.282759,0.000000,0.000000,2.219726,2.087609,1.526458,0.050693,1.079770,0.058960
4,1.631053,1.688322,0.546728,0.043189,0.376613,0.059539,0.221865,0.354969,0.280360,0.000000,...,0.080871,0.007650,0.000000,0.014182,2.010460,0.797082,0.029877,0.814288,0.028977,0.228851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12511,0.661059,0.304369,0.000000,0.007385,0.171296,0.161587,0.583334,1.431579,0.650368,0.514242,...,0.000000,0.011152,0.483963,0.125354,0.111080,0.005169,0.000000,0.113914,0.048215,0.718779
12512,0.174405,0.070080,0.369819,0.444969,0.798873,0.000000,0.002841,0.161540,2.776633,0.586198,...,0.000000,0.000000,0.530719,0.000000,0.041679,1.584406,0.235692,2.642931,0.000000,1.315126
12513,0.230343,0.501804,1.276318,0.015388,0.000000,0.020761,0.051612,0.009456,1.774340,0.000000,...,0.000000,0.026179,0.005674,0.081939,0.324825,1.174863,0.310520,0.765649,0.034178,1.013701
12514,0.152046,1.214670,0.430041,0.250063,0.835981,0.328676,0.000000,0.073166,0.999373,0.284025,...,0.020972,0.036839,0.562364,0.409505,1.634499,0.316930,0.288678,0.454185,0.550488,0.238925


In [4]:
train_labels

,image,category,original,generated
0,original/inpainting/14_original.png,0,1,0
1,generated/inpainting/14_inpainting.png,0,0,1
2,original/inpainting/22_original.png,0,1,0
3,generated/inpainting/22_inpainting.png,0,0,1
4,original/inpainting/29_original.png,0,1,0
...,...,...,...,...
12511,generated/style_transfer/Ukiyo_e_361_generated...,1,0,1
12512,original/style_transfer/Ukiyo_e_383_original.png,1,1,0
12513,generated/style_transfer/Ukiyo_e_383_generated...,1,0,1
12514,original/style_transfer/Ukiyo_e_384_original.png,1,1,0


In [5]:
train_data = pd.merge(train_labels, train_features, left_index=True, right_index=True)
test_data = pd.merge(test_labels, test_features, left_index=True, right_index=True)

In [10]:
def prepare_data_by_category(train_data, test_data, categories):
    train_data_by_category = train_data[train_data["category"].isin(categories)]
    test_data_by_category = test_data[test_data["category"].isin(categories)]
    X_train = train_data_by_category.iloc[:, 4:] # Feature sütunları
    y_train = train_data_by_category["generated"] # generated sütunu
    X_test = test_data_by_category.iloc[:, 4:]  # Feature sütunları
    y_test = test_data_by_category["generated"]  # generated sütunu
    return X_train, y_train, X_test, y_test

In [13]:
def scale(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [18]:
def train_model(X_train_scaled, y_train, X_test_scaled, y_test):
    #Train model
    model = LogisticRegression(max_iter=5000)
    model.fit(X_train_scaled, y_train)

    # Model performans
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    print("Model Accuracy:", accuracy)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [11]:
inpaintings_X_train, inpaintings_y_train, inpaintings_X_test, inpaintings_y_test = prepare_data_by_category(train_data, test_data, [0])
style_transfer_X_train, style_transfer_y_train, style_transfer_X_test, style_transfer_y_test  = prepare_data_by_category(train_data, test_data, [1])
combined_X_train, combined_y_train, combined_X_test, combined_y_test = prepare_data_by_category(train_data, test_data, [0, 1])

In [14]:
inpaintings_X_train_scaled, inpaintings_X_test_scaled = scale(inpaintings_X_train, inpaintings_X_test)
style_transfer_X_train_scaled, style_transfer_X_test_scaled = scale(style_transfer_X_train, style_transfer_X_test)
combined_X_train_scaled, combined_X_test_scaled = scale(combined_X_train, combined_X_test)

In [19]:
train_model(inpaintings_X_train_scaled, inpaintings_y_train, inpaintings_X_test_scaled, inpaintings_y_test)
train_model(style_transfer_X_train_scaled, style_transfer_y_train, style_transfer_X_test_scaled, style_transfer_y_test)
train_model(combined_X_train_scaled, combined_y_train, combined_X_test_scaled, combined_y_test)

Model Accuracy: 0.7870597870597871

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.78      0.79      1221
           1       0.79      0.79      0.79      1221

    accuracy                           0.79      2442
   macro avg       0.79      0.79      0.79      2442
weighted avg       0.79      0.79      0.79      2442

Model Accuracy: 0.9756838905775076

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       658
           1       0.97      0.98      0.98       658

    accuracy                           0.98      1316
   macro avg       0.98      0.98      0.98      1316
weighted avg       0.98      0.98      0.98      1316

Model Accuracy: 0.8347525279403938

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.82      0.83      1879
           1       0.83      0.85      0.84      1879

  